<h2>KMeans</h2>

<ul>
    <li>This code snippet sets up a PySpark environment in a Python script.</li><li> It first imports the necessary modules like pyspark, os, and sys.</li><li> Then, it sets the Python executable for PySpark to the same one being used by the script.</li><li> Finally, it imports the SparkContext class for creating RDDs and the SparkSession class for programming Spark with the DataFrame API.</li>
    </ul>

In [ ]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession

<ul><li>This code creates a SparkSession named spark with specific configuration options. It sets the driver memory to 16 GB and names the application 'chapter_5'.</li><ul>

In [ ]:
spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_5').getOrCreate()

<h3>A First Take on Clustering</h3>

<ul><li>This code reads a CSV file named "kddcup.data_10_percent_corrected" into a DataFrame data_without_header without inferring the schema and without treating the first row as a header.</li><li> It then assigns the column names to the DataFrame based on a predefined list column_names. </li><li>Finally, it creates a new DataFrame data with the specified column names by using the toDF(*column_names) method.</li></ul>


In [ ]:
data_without_header = spark.read.option("inferSchema", True).\option("header", False).\csv("data/kddcup.data_10_percent_corrected")
column_names = [ "duration", "protocol_type", "service", "flag",
 "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
 "hot", "num_failed_logins", "logged_in", "num_compromised",
 "root_shell", "su_attempted", "num_root", "num_file_creations",
 "num_shells", "num_access_files", "num_outbound_cmds",
 "is_host_login", "is_guest_login", "count", "srv_count",
 "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
 "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
 "dst_host_count", "dst_host_srv_count",
 "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
 "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
 "dst_host_serror_rate", "dst_host_srv_serror_rate",
 "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
 "label"]
data = data_without_header.toDF(*column_names)

<ul><li>
This code snippet selects the "label" column from the DataFrame data, groups the data by "label", calculates the count of each group, and then orders the result in descending order based on the count.</li></ul>

In [ ]:
from pyspark.sql.functions import col
data.select("label").groupBy("label").count().\
 orderBy(col("count").desc()).show(25)

<ul><li>This code performs KMeans clustering on the numeric features of the data DataFrame. It drops non-numeric columns, assembles the numeric features into a single feature vector, creates a KMeans estimator, constructs a pipeline, fits the pipeline to the data, and then prints the cluster centers.</li></ul>

In [ ]:
from pyspark.ml.feature importVectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml import Pipeline
numeric_only= data.drop("protocol_type", "service","flag").cache()
assembler= VectorAssembler().setInputCols(numeric_only.columns[:-1]).\
 setOutputCol("featureVector")
kmeans =KMeans().setPredictionCol("cluster").setFeaturesCol("featureVector")
pipeline= Pipeline().setStages([assembler, kmeans])
pipeline_model= pipeline.fit(numeric_only)
kmeans_model= pipeline_model.stages[1]
from pprint import pprint
pprint(kmeans_model.clusterCenters())

<ul><li>This code uses the fitted pipeline model to add a "cluster" column to the numeric_only DataFrame, indicating the cluster to which each row belongs. It then groups the data by "cluster" and "label", calculates the count of each group and then displays it.</li></ul>

In [ ]:
with_cluster =pipeline_model.transform(numeric_only)
with_cluster.select("cluster", "label").groupBy("cluster", "label").count().\
 orderBy(col("cluster"), col("count").desc()).show(25)

<h3>Choosing k</h3>

<b>This function calculates the clustering score for a given dataset and number of clusters.</b>
<ul><li>It drops non-numeric columns, sets up a VectorAssembler, creates a KMeans estimator with a random seed and the specified number of clusters, constructs a pipeline, fits the pipeline to the data, and computes the training cost of the KMeans model.</li><li> It then iterates over a range of k values and prints the clustering score for each k.</li></ul>

In [ ]:
from pyspark.sql import DataFrame
from random import randint
def clustering_score(input_data, k):
    input_numeric_only = input_data.drop("protocol_type", "service", "flag")
    assembler = VectorAssembler().setInputCols(input_numeric_only.columns[:-1]).setOutputCol("featureVector")
    kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setPredictionCol("cluster").setFeaturesCol("featureVector")
    pipeline = Pipeline().setStages([assembler, kmeans])
    pipeline_model = pipeline.fit(input_numeric_only)
    kmeans_model = pipeline_model.stages[-1]
    training_cost = kmeans_model.summary.trainingCost
    return training_cost
for k in list(range(20,100, 20)):
    print(clustering_score(numeric_only, k))

<b>This updated function clustering_score_1(input_data, k) is similar to the previous one but with additional settings for the KMeans estimator.</b><ul><li> It sets a maximum number of iterations (setMaxIter), a tolerance value (setTol), and then fits the pipeline to the input data to calculate the training cost. </li><li>The function then iterates over a range of k values (20 to 100, with a step of 20) and prints the clustering score for each k value.</li>

In [ ]:
def clustering_score_1(input_data, k):
    input_numeric_only = input_data.drop("protocol_type", "service", "flag")
    assembler = VectorAssembler().\
 setInputCols(input_numeric_only.columns[:-1]).\
 setOutputCol("featureVector")
    kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setMaxIter(40).\
 setTol(1.0e-5).\
 setPredictionCol("cluster").setFeaturesCol("featureVector")
    pipeline = Pipeline().setStages([assembler, kmeans])
    pipeline_model = pipeline.fit(input_numeric_only)
    kmeans_model = pipeline_model.stages[-1]
    training_cost = kmeans_model.summary.trainingCost
    return training_cost
for k in list(range(20,101, 20)):
    print(k, clustering_score_1(numeric_only, k))

<h3>Feature Normalization</h3>

<b>This function clustering_score_2(input_data, k) is similar to clustering_score_1 but includes an additional step to standardize the features using StandardScaler. </b><ul><li>It sets up the StandardScaler with input and output columns and parameters for standardization.</li><li> The function then constructs a pipeline with the VectorAssembler, StandardScaler, and KMeans estimator, fits the pipeline to the data, and calculates the training cost for each k value in the specified range (60 to 270, with a step of 30).</li></ul>

In [ ]:
from pyspark.ml.feature import StandardScaler
def clustering_score_2(input_data, k):
    input_numeric_only = input_data.drop("protocol_type", "service", "flag")
    assembler = VectorAssembler().\
 setInputCols(input_numeric_only.columns[:-1]).\
 setOutputCol("featureVector")
    scaler = StandardScaler().setInputCol("featureVector").\
 setOutputCol("scaledFeatureVector").\
 setWithStd(True).setWithMean(False)
    kmeans = KMeans().setSeed(randint(100,100000)).\
 setK(k).setMaxIter(40).\
 setTol(1.0e-5).setPredictionCol("cluster").\
 setFeaturesCol("scaledFeatureVector")
    pipeline = Pipeline().setStages([assembler, scaler, kmeans])
    pipeline_model = pipeline.fit(input_numeric_only)
    kmeans_model = pipeline_model.stages[-1]
    training_cost = kmeans_model.summary.trainingCost
    return training_cost
for k in list(range(60, 271, 30)):
    print(k, clustering_score_2(numeric_only, k))

<h3>Categorical Valus</h3>

<b>This function one_hot_pipeline(input_col) creates a pipeline for one-hot encoding a categorical column.</b><ul><li> It takes an input column name, creates a StringIndexer to convert the categorical column into numerical indices, and then applies a OneHotEncoder to convert the indices into a sparse vector.</li><li> The function returns the pipeline and the name of the output column containing the one-hot encoded vector.</li></ul>

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
def one_hot_pipeline(input_col):
    indexer = StringIndexer().setInputCol(input_col).setOutputCol(input_col + "_indexed")
    encoder = OneHotEncoder().setInputCol(input_col + "_indexed").setOutputCol(input_col + "_vec")
    pipeline = Pipeline().setStages([indexer, encoder])
    return pipeline, input_col + "_vec"

<b>This function clustering_score_3(input_data, k) extends the previous clustering functions by incorporating one-hot encoding for categorical columns "protocol_type," "service," and "flag."</b><ul><li> It uses the one_hot_pipeline function to create pipelines for each categorical column.</li><li> It then assembles all relevant columns (excluding "label" and the original categorical columns) into a feature vector, scales the features, and applies KMeans clustering.</li><li> The function iterates over a range of k values and prints the training cost for each k.</li></ul>

In [ ]:
def clustering_score_3(input_data, k):
    proto_type_pipeline, proto_type_vec_col = one_hot_pipeline("protocol_type")
    service_pipeline, service_vec_col = one_hot_pipeline("service")
    flag_pipeline, flag_vec_col = one_hot_pipeline("flag")
    assemble_cols = set(input_data.columns)- \
 {"label", "protocol_type", "service", "flag"} | \
 {proto_type_vec_col, service_vec_col, flag_vec_col}
    assembler = VectorAssembler().setInputCols(list(assemble_cols)).setOutputCol("featureVector")
    scaler = StandardScaler().setInputCol("featureVector").setOutputCol("scaledFeatureVector").setWithStd(True).setWithMean(False)
    kmeans = KMeans().setSeed(randint(100,100000)).setK(k).setMaxIter(40).setTol(1.0e-5).setPredictionCol("cluster").setFeaturesCol("scaledFeatureVector")
    pipeline = Pipeline().setStages([proto_type_pipeline, service_pipeline,flag_pipeline, assembler, scaler, kmeans])
    pipeline_model = pipeline.fit(input_data)
    kmeans_model = pipeline_model.stages[-1]
    training_cost = kmeans_model.summary.trainingCost
    return training_cost
for k in list(range(60, 271, 30)):
    print(k, clustering_score_3(data, k)

<b>This function entropy(counts) calculates the entropy of a given list of counts.</b><ul><li> It first filters out counts that are greater than zero, calculates the total count n, and computes the probabilities p for each count.</li></ul> <b> Finally, it calculates the entropy using the formula Σ(-p * log(p)) and returns the result.</b>

In [ ]:
from math import log
def entropy(counts):
    values = [c for c in counts if (c > 0)]
    n = sum(values)
    p = [v/n for v in values]
    return sum([-1*(p_v) * log(p_v) for p_v in p])

<b>This code calculates the weighted average entropy of each cluster in a clustering model.</b><ul><li> It first transforms the data using the fitted pipeline model to assign each data point to a cluster.</li><li> Then, it groups the data by "cluster" and "label" and counts the occurrences, ordering the result by "cluster." </li><li>It computes the probability p_col for each label within each cluster using a window function to calculate the sum of counts per cluster.</li><li> Next, it calculates the entropy for each cluster by aggregating the sum of the product of p_col and its logarithm.</li><li> Finally, it computes the weighted average entropy by multiplying the entropy of each cluster by its size, summing these values, and dividing by the total number of data points.</li></ul>

In [ ]:
from pyspark.sql import functions as fun
from pyspark.sql import Window
cluster_label = pipeline_model.transform(data).select("cluster", "label")
df = cluster_label.\
 groupBy("cluster", "label").\
 count().orderBy("cluster")
w = Window.partitionBy("cluster")
p_col = df['count'] / fun.sum(df['count']).over(w)
with_p_col = df.withColumn("p_col", p_col)
result = with_p_col.groupBy("cluster").\
 agg((-fun.sum(col("p_col") * fun.log2(col("p_col"))))
 .alias("entropy"),
 fun.sum(col("count"))
 .alias("cluster_size"))
result = result.withColumn('weightedClusterEntropy',fun.col('entropy') * fun.col('cluster_size'))
weighted_cluster_entropy_avg = result.\
 agg(fun.sum(
 col('weightedClusterEntropy'))).\
 collect()
weighted_cluster_entropy_avg[0][0]/data.count()

<b>This function fit_pipeline_4(data, k) creates and fits a pipeline for KMeans clustering with one-hot encoding for categorical columns "protocol_type," "service," and "flag."</b><ul><li> It first uses the one_hot_pipeline function to create pipelines for each categorical column. </li><li>Then, it assembles all relevant columns (excluding "label" and the original categorical columns) into a feature vector, scales the features, and applies KMeans clustering. Finally, it returns the fitted pipeline model.</li></ul>

In [ ]:
def fit_pipeline_4(data, k):
    (proto_type_pipeline, proto_type_vec_col) = one_hot_pipeline("protocol_type")
    (service_pipeline, service_vec_col) = one_hot_pipeline("service")
    (flag_pipeline, flag_vec_col) = one_hot_pipeline("flag")
    assemble_cols = set(data.columns)- {"label", "protocol_type", "service", "flag"} | {proto_type_vec_col, service_vec_col, flag_vec_col}
    assembler = VectorAssembler(inputCols=list(assemble_cols), outputCol="featureVector")
    scaler = StandardScaler(inputCol="featureVector", outputCol="scaledFeatureVector", withStd=True, withMean=False)
    kmeans = KMeans(seed=randint(100, 100000), k=k, predictionCol="cluster", featuresCol="scaledFeatureVector", maxIter=40, tol=1.0e-5)
    pipeline = Pipeline(stages=[proto_type_pipeline, service_pipeline, flag_pipeline, assembler, scaler, kmeans])
    return pipeline.fit(data)

<b>This function clustering_score_4(input_data, k) calculates the weighted average entropy of clusters for a given dataset and number of clusters.</b><ul><li> It first fits a pipeline using fit_pipeline_4 to transform the data and assign each data point to a cluster.</li><li> Then, it calculates the entropy for each cluster by grouping the data by "cluster" and "label," and computing the probability p_col for each label within each cluster. </li><li>Next, it calculates the entropy for each cluster and computes the weighted cluster entropy by multiplying the entropy of each cluster by its size. </li><li>Finally, it computes the weighted average entropy by summing the weighted cluster entropies and dividing by the total number of data points.</li></ul>

In [ ]:
def clustering_score_4(input_data, k):
    pipeline_model = fit_pipeline_4(input_data, k)
    cluster_label = pipeline_model.transform(input_data).select("cluster", "label")
    df = cluster_label.groupBy("cluster", "label").count().orderBy("cluster")
    w = Window.partitionBy("cluster")
    p_col = df['count'] / fun.sum(df['count']).over(w)
    with_p_col = df.withColumn("p_col", p_col)
    result = with_p_col.groupBy("cluster").agg(-fun.sum(col("p_col") * fun.log2(col("p_col"))).alias("entropy"),fun.sum(col("count")).alias("cluster_size"))
    result = result.withColumn('weightedClusterEntropy', col('entropy') * col('cluster_size'))
    weighted_cluster_entropy_avg = result.agg(fun.sum(col('weightedClusterEntropy'))).collect()
    return weighted_cluster_entropy_avg[0][0] / input_data.count()

<h3>Clustering in Action</h3>

<b>This code fits a pipeline with 180 clusters using the fit_pipeline_4 function and then transforms the data using the fitted pipeline to assign each data point to a cluster.</b><ul><li> It then selects the "cluster" and "label" columns, groups the data by "cluster" and "label," counts the occurrences, and orders the result by "cluster" and "label."</li><li> Finally, it shows the resulting DataFrame with counts for each cluster-label combination.</li></ul>

In [ ]:
pipeline_model= fit_pipeline_4(data, 180)
count_by_cluster_label = pipeline_model.transform(data).\select("cluster", "label").\groupBy("cluster", "label").\count().orderBy("cluster", "label")
count_by_cluster_label.show()